In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import networkx as nx
from scipy.sparse import lil_matrix, csr_matrix, vstack
import copy
import os
from wl import WL_compute, WL_compute_original

In [3]:
if 0 == 1:
    for start, end in [(0, 7)]:
        print("# YES", test_graph_names[start:end])
        k, _, label_lookup = get_wl_for_graph_names(test_graph_names[start:end], fn = WL_compute)
        k_org, _ = get_wl_for_graph_names(test_graph_names[start:end], fn = WL_compute_original)
        print(k_org[-1] == k[-1])

## Retrieve graphs from Tobias' concept-graph extraction library

In [4]:
import networkx as nx
import matplotlib.pyplot as plt
from glob import glob
GRAPH_DIR = '../falke/code/data/ng20/{}_graphs'

def get_graphs(directory):
    graphs = {}
    for topic in os.listdir(directory):
        graph_dir = os.path.join(directory, topic)
        if not os.path.isdir(graph_dir): continue
        graph_file = glob(os.path.join(graph_dir, '*.gml'))[0]
        with open(graph_file) as f:
            graph = f.read().split('\n')
        for idx, line in enumerate(graph):
            if line.startswith('label'):
                next_line = graph[idx + 1]
                label = next_line.replace('name', 'label')
                graph[idx] = label
        new_graph_file = graph_file.replace('.gml', '.renamed.gml')
        with open(new_graph_file, 'w') as f:
            f.write('\n'.join(graph))
        graph = nx.read_gml(new_graph_file, label = 'label')
        if graph.number_of_nodes() > 0 and graph.number_of_edges() > 0:
            graphs[topic] = graph
        else:
            graphs[topic] = None
            print("Found empty graph: {}".format(graph_file))
    return graphs
graphs_test_docs = get_graphs(GRAPH_DIR.format('test'))
graphs_train_docs = get_graphs(GRAPH_DIR.format('train'))
assert graphs_test_docs.keys() == graphs_train_docs.keys()
assert len(graphs_test_docs.keys()) > 0

all_nodes = [val for sublist in [g.nodes() for g in list(graphs_train_docs.values()) + list(graphs_test_docs.values()) if g is not None] for val in sublist]

Found empty graph: ../falke/code/data/ng20/test_graphs/rec.sport.baseball/ng-20.graph.gml
Found empty graph: ../falke/code/data/ng20/test_graphs/sci.med/ng-20.graph.gml
Found empty graph: ../falke/code/data/ng20/test_graphs/sci.space/ng-20.graph.gml
Found empty graph: ../falke/code/data/ng20/test_graphs/talk.politics.guns/ng-20.graph.gml
Found empty graph: ../falke/code/data/ng20/test_graphs/talk.politics.misc/ng-20.graph.gml


## Some helper

In [5]:
def get_wl_for_graphs(graphs, all_nodes = all_nodes,  h = 10, fn = WL_compute):
    nodes = [g.nodes() for g in graphs]
    adjs = [nx.adjacency_matrix(g).toarray() for g in graphs]
    return fn(adjs, nodes, all_nodes = all_nodes, h = h)

def get_wl_for_graph_names(graph_names, add_all_nodes = False, h = 10, fn = WL_compute):
    graphs = [graphs_test_docs[x] for x in graph_names]
    graphs[0] = graphs[0].copy()
    if add_all_nodes:
        for node in all_nodes:
            graphs[0].add_node(node)
    return get_wl_for_graphs(graphs, all_nodes, h = h, fn = fn)

test_graph_names = list(sorted(graphs_test_docs.keys()))

## Calculate gram-matrix for WL kernel for the training concept-graphs

In [6]:
#%%time
from sklearn import svm

augmented_graph = graphs_train_docs[list(graphs_train_docs.keys())[0]]
#for node in all_nodes:
#    augmented_graph.add_node(node)
adjs = []
nodes = []
for topic, graph in sorted(graphs_train_docs.items(), key = lambda x: x[0]):
    adjs.append(nx.adjacency_matrix(graph).toarray())
    nodes.append(graph.nodes())
print('Calculating WL_train: start')
K_train, phi_list_train, label_lookup_train = WL_compute(adjs, nodes, 2, all_nodes = all_nodes, DEBUG = True)
print('Calculating WL_train: end')

Calculating WL_train: start
Number of original labels 5529
K original is computed
Iteration 0: phi is computed
Number of compressed labels 5227
Itaration 0: phi sparse saved
Iteration 0: K is computed
Iteration 1: phi is computed
Number of compressed labels 5227
Itaration 1: phi sparse saved
Iteration 1: K is computed
Calculating WL_train: end


In [16]:
print(K_train[-1].shape)
len(graphs_train_docs.keys())

(20, 20)


20

## Train SVM classifier on the gram-matrix

In [7]:
clf = svm.SVC(kernel = 'precomputed', verbose = True, probability=True, )
clf.fit(K_train[-1], sorted(graphs_train_docs.keys()))

[LibSVM]

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto',
  kernel='precomputed', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=True)

## Test classifier

In [12]:

#%%time
all_graphs = [('test', topic, graph) for topic, graph in graphs_test_docs.items()]
all_graphs = all_graphs + [('train', topic, graph) for topic, graph in graphs_train_docs.items()]
for set_, test_topic, graph in sorted(all_graphs, key = lambda x: x[0]):
    if graph is None: continue
    print('Testing graph:\t{:40} ({})'.format(test_topic, set_))
    test_graph = graph
    adjs_test = [nx.adjacency_matrix(test_graph).toarray()] + adjs
    nodes_test = [test_graph.nodes()] + nodes
    K_test, phi_list_test, label_lookup_test = WL_compute(adjs_test, nodes_test, 2, all_nodes = all_nodes, DEBUG = False)
    X_test = K_test[-1][0, 1:].reshape(1, -1)
    print('Predicted:\t{}'.format(clf.predict(X_test)[0]))
    probs_test = clf.predict_proba(X_test)[0]
    probs_zipped = list(zip(clf.classes_, probs_test))
    for clazz, prob in sorted(probs_zipped, key = lambda x: x[1])[:5]:
        print('\t{:.3f}\t{}'.format(prob, clazz))
    print()

Testing graph:	alt.atheism                              (test)
Predicted:	rec.motorcycles
	0.037	rec.motorcycles
	0.039	misc.forsale
	0.040	comp.sys.ibm.pc.hardware
	0.040	alt.atheism
	0.042	comp.sys.mac.hardware

Testing graph:	comp.windows.x                           (test)
Predicted:	rec.motorcycles
	0.037	rec.motorcycles
	0.039	misc.forsale
	0.040	comp.sys.ibm.pc.hardware
	0.040	alt.atheism
	0.042	comp.sys.mac.hardware

Testing graph:	talk.politics.mideast                    (test)
Predicted:	rec.motorcycles
	0.037	rec.motorcycles
	0.039	misc.forsale
	0.040	comp.sys.ibm.pc.hardware
	0.040	alt.atheism
	0.042	comp.sys.mac.hardware

Testing graph:	comp.sys.ibm.pc.hardware                 (test)
Predicted:	rec.motorcycles
	0.037	rec.motorcycles
	0.039	misc.forsale
	0.040	comp.sys.ibm.pc.hardware
	0.040	alt.atheism
	0.042	comp.sys.mac.hardware

Testing graph:	talk.religion.misc                       (test)
Predicted:	rec.motorcycles
	0.037	rec.motorcycles
	0.039	misc.forsale
	0.040	comp

## Draw graphs

In [9]:
if 0 == 1:
    fig = plt.figure(figsize=(30, 30))
    pos=nx.spring_layout(g_mult_dir)
    nx.draw(g_mult_dir, pos = pos)

    edge_labels=dict([((u,v,),d['name'])
                 for u,v,d in g_mult_dir.edges(data=True)])
    nx.draw_networkx_edge_labels(g_mult_dir,pos,edge_labels=edge_labels)
    nx.draw_networkx_nodes(g_mult_dir, pos = pos, label='name', )
    nx.draw_networkx_labels(g_mult_dir, pos = pos)
    plt.show()

    graphs = sorted(list(nx.weakly_connected_component_subgraphs(g_mult_dir)), key = len)#, reverse = True)

    for graph in graphs[-4:]:
        fig = plt.figure(figsize=(20, 20))
        pos=nx.spring_layout(graph)
        #nx.draw(graph, pos = pos)
        edge_labels=dict([((u,v,),d['name'])
                     for u,v,d in graph.edges(data=True)])
        nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels)
        nx.draw_networkx_edges(graph, pos, arrows = True)
        nx.draw_networkx_nodes(graph, pos = pos, label='name')
        nx.draw_networkx_labels(graph, pos = pos)
        plt.show()
